In [1]:
import json

def load_json_data(file):
    with open(file) as json_data:
        data = json.load(json_data)
    return data

In [2]:
# Path
file = '/home/jonas/git/knowledge-probing-private/data/outputs/probe_bert/bert_only_probe_for_values_bert-base-uncased_trained-False_pre-trained_12_3_2021__14-2/model_data.json'


certainty_data = load_json_data(file)

In [3]:
line = [1,2,3]
two = [3,4,5]
final = []
final.append(line)
final.append(two)
print(final)

[[1, 2, 3], [3, 4, 5]]


In [4]:
import plotly.graph_objects as go
import numpy as np


def make_line_plot(lines, correct_lines, wrong_lines):
    general_font_size = 20
    font_size_lengend = 25

    fig = go.Figure()

    # Sample data
    x = np.arange(len(lines[0][0]))

    
    for data in wrong_lines:
        # print(data)
        line, rank = data
        # print(line)
        # For every data series, just add a trace
        fig.add_trace(go.Scatter(mode='lines+markers', x=x, y=line, line_color='black', marker=dict(
                                                                                        color="black",
                                                                                        symbol='circle',
                                                                                        size=10)))


    for data in correct_lines:
        # print(data)
        line, rank = data
        # print(line)
        # For every data series, just add a trace
        fig.add_trace(go.Scatter(mode='lines+markers', x=x, y=line, line_color='coral', marker=dict(
                                                                                        color="lightcoral",
                                                                                        symbol='triangle-down',
                                                                                        size=10)))


    # for data in lines:
    #     # print(data)
    #     line, rank = data
    #     # print(line)
    #     # For every data series, just add a trace
    #     fig.add_trace(go.Scatter(mode='lines+markers', x=x, y=line))


    fig.update_layout(xaxis_title='',
                        yaxis_title='',
                        autosize=True,
                        font=dict(
                            family="Courier New, monospace",
                            size=general_font_size,  # default for
                            color="black"
                        ),
                        # yaxis_title='{}'.format(nice_metric_names[metric])
                        showlegend=True, plot_bgcolor='rgba(0,0,0,0)',
                        legend=dict(x=1, y=1,
                                    font=dict(
                                        family="sans-serif",
                                        size=font_size_lengend,
                                        color="black"
                                    ),
                                    bgcolor="rgba(255,255,255,0)",
                                    ))


    fig.update_xaxes(showline=True, linewidth=1,
                    linecolor='black', showgrid=True, gridcolor='rgba(200,200,200,0.4)')
    fig.update_yaxes(showline=True, linewidth=1,
                    linecolor='black', showgrid=True, gridcolor='rgba(200,200,200,0.4)')

    fig.show()

In [19]:
import plotly.figure_factory as ff
import numpy as np


def make_hist_plot(correct, wrong): 

    hist_data = [correct, wrong]

    group_labels = ['Correct', 'Wrong']
    colors = ['green', 'red']

    # Create distplot with curve_type set to 'normal'
    fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

    # Add title
    fig.update_layout(title_text='Softmax probabilities of correct and wrong predictions', xaxis_title='Softmax probability of the highest ranked token [%]')
    fig.show()

In [20]:
def analyze_data(individual_predictions):
    num_instances = len(individual_predictions)
    # analyze if the difference between the first and the second value is bigger for correct predictions

    firsts_correct = []
    firsts_wrong = []

    num_correct = 0
    num_wrong = 0

    sum_first_correct = 0
    sum_first_wrong = 0

    list_diffs_correct = []
    list_diffs_wrong = []

    diffs_correct = 0
    diffs_wrong = 0

    for pred in individual_predictions:
        # print(pred)
        values = pred['top_k_values'][:10]
        rank = pred['rank']
        # print(values)

        diff = values[0]/values[1] 
        
        if rank == 0:
            firsts_correct.append(values[0])
            list_diffs_correct.append(diff)
            sum_first_correct = sum_first_correct + values[0]
            diffs_correct = diffs_correct + diff
            num_correct = num_correct + 1
        else:
            firsts_wrong.append(values[0])
            list_diffs_wrong.append(diff)
            sum_first_wrong = sum_first_wrong + values[0]
            diffs_wrong = diffs_wrong + diff
            num_wrong = num_wrong + 1


        # print(values[0]/values[1])

    if num_correct > 0:

        mean_first_correct = sum_first_correct / num_correct
        mean_first_wrong = sum_first_wrong / num_wrong

        mean_diff_correct = diffs_correct / num_correct
        mean_diff_wrong = diffs_wrong / num_wrong

        print('Num correct: ', num_correct)
        print('Num wrong: ', num_wrong)

        print('First correct, false')
        print(mean_first_correct)
        print(mean_first_wrong)
        print('Diffs correct, wrong')
        print(mean_diff_correct)
        print(mean_diff_wrong)
    else:
        print('No correct answers')

    make_hist_plot(firsts_correct, firsts_wrong)

    # make_hist_plot(list_diffs_correct, list_diffs_wrong)

In [21]:
print(certainty_data.keys())

for dataset in certainty_data.keys():
    print('Dataset: ', dataset)

    dataset_individual_predictions = []

    for relation in certainty_data[dataset].keys():
        if 'means' in relation:
            continue 
        print('Relation: ', relation)
        num_elements = len(certainty_data[dataset][relation][0]['individual_predictions'])
        print('num: ', num_elements)


        all_lines = []
        correct_lines = []
        wrong_lines = []

        dataset_individual_predictions.extend(certainty_data[dataset][relation][0]['individual_predictions'])

        for i, instance in enumerate(certainty_data[dataset][relation][0]['individual_predictions']):
            # print(instance)
            if i > 549:
                break
            # print(instance['top_k_values'][:10])
            # all_lines.append((instance['top_k_values'][:10], instance['rank']))
            if instance['rank'] == 0:
                correct_lines.append((instance['top_k_values'][:10], instance['rank']))
            else: 
                wrong_lines.append((instance['top_k_values'][:10], instance['rank']))
            # print(instance['rank'])

        individual_predictions = certainty_data[dataset][relation][0]['individual_predictions']
        # analyze_data(individual_predictions=individual_predictions)
        print('\n\n')
        # make_line_plot(lines, correct_lines, wrong_lines)
        
    analyze_data(individual_predictions=dataset_individual_predictions)


dict_keys(['Google_RE', 'TREx', 'ConceptNet', 'Squad'])
Dataset:  Google_RE
Relation:  place_of_birth
num:  2937



Relation:  date_of_birth
num:  1825



Relation:  place_of_death
num:  766



Num correct:  569
Num wrong:  4959
First correct, false
11.494424639770562
9.923207191960373
Diffs correct, wrong
1.1314248986423039
1.0659571686179463


Dataset:  TREx
Relation:  P19
num:  944



Relation:  P20
num:  953



Relation:  P279
num:  964



Relation:  P37
num:  966



Relation:  P413
num:  952



Relation:  P449
num:  880



Relation:  P47
num:  920



Relation:  P138
num:  642



Relation:  P364
num:  856



Relation:  P463
num:  225



Relation:  P101
num:  696



Relation:  P106
num:  958



Relation:  P527
num:  976



Relation:  P530
num:  996



Relation:  P176
num:  973



Relation:  P27
num:  966



Relation:  P407
num:  877



Relation:  P30
num:  975



Relation:  P178
num:  591



Relation:  P1376
num:  233



Relation:  P131
num:  881



Relation:  P1412
num:  969



Relation:  P108
num:  383



Relation:  P136
num:  931



Relation:  P17
num:  930



Relation:  P39
num:  892



Relation:  P264
num:  429



Relation:  P276
num:  958



Relation:  P937
num:  954



Relation:  P140
num:  473



Relation:  P1303
num:  949



Relation:  P127
num:  687



Relation:  P103
num:  977



Relation:  P190
num:  992



Rela

Dataset:  ConceptNet
Relation:  test
num:  12514



Num correct:  1892
Num wrong:  10622
First correct, false
12.741473971411239
10.175113801699398
Diffs correct, wrong
1.1805633915141398
1.0995025452238574


Dataset:  Squad
Relation:  test
num:  305



Num correct:  40
Num wrong:  265
First correct, false
12.34571213722229
10.234460168514612
Diffs correct, wrong
1.1110605736056853
1.0931228878390369


In [22]:
# print(len(individual_predictions))

    # break